# Welcome to Christian's Jupyter Public Notebook

Segmenting and Clustering Neighborhoods in Toronto

# First part of the exercise

In [1]:
import pandas as pd
import numpy as py
import pip 

Reading content from Wikipedia into data frame. The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood

In [2]:
m = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M', match='Postal Code')
mdf = m.pop(0)
mdf = mdf.rename(columns={'Postal Code': 'PostalCode', 'Neighbourhood': 'Neighborhood'})
mdf.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.

In [3]:
mdf_c=mdf[mdf.Borough!="Not assigned"].reset_index(drop=True)

More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.

In [4]:
test_postal_code = mdf_c.loc[mdf_c['PostalCode'] == 'M5A']
test_postal_code

,PostalCode,Borough,Neighborhood
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"


If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.

In [6]:
mdf_cr = mdf_c.groupby(["PostalCode", "Borough"], as_index=False).agg(lambda x: ", ".join(x))
mdf_cr["Neighborhood"]=mdf_cr["Neighborhood"].replace("Not assigned",mdf_cr["Borough"])

In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.

In [7]:
mdf_cr.shape

(103, 3)

Final check to compare exercise table to created data set

In [12]:
list = ["M5G", "M2H", "M4B", "M1J", "M4G", "M4M", "M1R", "M9V", "M9L", "M5V", "M1B", "M5A"]
df=pd.DataFrame(columns=mdf_cr.columns)
for postalcode in list:
    df=df.append(mdf_cr[mdf_cr["PostalCode"]==postalcode],ignore_index=True)
df

,PostalCode,Borough,Neighborhood
0,M5G,Downtown Toronto,Central Bay Street
1,M2H,North York,Hillcrest Village
2,M4B,East York,"Parkview Hill, Woodbine Gardens"
3,M1J,Scarborough,Scarborough Village
4,M4G,East York,Leaside
5,M4M,East Toronto,Studio District
6,M1R,Scarborough,"Wexford, Maryvale"
7,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest..."
8,M9L,North York,Humber Summit
9,M5V,Downtown Toronto,"CN Tower, King and Spadina, Railway Lands, Har..."


# Second part of the exercise

In [18]:
url="http://cocl.us/Geospatial_data"
coor=pd.read_csv(url)
coor = coor.rename(columns={'Postal Code': 'PostalCode'})
mdf_coor=pd.merge(mdf_cr,coor,how='inner',on="PostalCode")
mdf_coor.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
